# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/12_vision_avancee/12_demo_object_detection.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '12_demo_object_detection.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 13 - Détection d'Objets avec YOLO

Ce notebook explore la **détection d'objets** avec les architectures YOLO (You Only Look Once).

## Objectifs
- Comprendre les architectures de détection (R-CNN vs YOLO)
- Utiliser YOLOv5 et YOLOv8 pour la détection temps réel
- Entraîner un détecteur sur un dataset custom
- Évaluer avec les métriques IoU, mAP
- Implémenter Non-Maximum Suppression (NMS)

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.ops import nms, box_iou
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import requests
from io import BytesIO
import time

# YOLOv8 (Ultralytics)
try:
    from ultralytics import YOLO
    YOLO_AVAILABLE = True
except ImportError:
    print("⚠️ ultralytics not installed. Install with: pip install ultralytics")
    YOLO_AVAILABLE = False

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Fondamentaux : IoU et NMS

### 1.1 Intersection over Union (IoU)

In [ ]:
def compute_iou(box1, box2):
    """
    Calcule l'IoU entre deux bounding boxes.
    
    Args:
        box1, box2: [x1, y1, x2, y2] format (coin sup gauche, coin inf droit)
    
    Returns:
        IoU score (0-1)
    """
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    
    # Aire de l'intersection
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    inter_area = inter_width * inter_height
    
    # Aires des boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Union = box1 + box2 - intersection
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

# Exemple
box_gt = [50, 50, 150, 150]  # Ground truth
box_pred1 = [60, 60, 160, 160]  # Bonne prédiction
box_pred2 = [200, 200, 300, 300]  # Mauvaise prédiction

iou1 = compute_iou(box_gt, box_pred1)
iou2 = compute_iou(box_gt, box_pred2)

print(f"IoU (bonne détection): {iou1:.3f}")
print(f"IoU (mauvaise détection): {iou2:.3f}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, box_pred, iou, title in zip(axes, [box_pred1, box_pred2], [iou1, iou2], 
                                     ['Bonne détection', 'Mauvaise détection']):
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 400)
    ax.invert_yaxis()
    ax.set_aspect('equal')
    
    # Ground truth (vert)
    rect_gt = patches.Rectangle((box_gt[0], box_gt[1]), 
                                box_gt[2]-box_gt[0], box_gt[3]-box_gt[1],
                                linewidth=2, edgecolor='green', facecolor='none', label='Ground Truth')
    ax.add_patch(rect_gt)
    
    # Prédiction (rouge)
    rect_pred = patches.Rectangle((box_pred[0], box_pred[1]), 
                                  box_pred[2]-box_pred[0], box_pred[3]-box_pred[1],
                                  linewidth=2, edgecolor='red', facecolor='none', label='Prediction')
    ax.add_patch(rect_pred)
    
    ax.set_title(f'{title}\nIoU = {iou:.3f}')
    ax.legend()

plt.tight_layout()
plt.show()

### 1.2 Non-Maximum Suppression (NMS)

NMS élimine les détections redondantes (plusieurs boxes pour le même objet).

In [ ]:
def non_max_suppression(boxes, scores, iou_threshold=0.5):
    """
    Applique Non-Maximum Suppression.
    
    Args:
        boxes: Tensor [N, 4] de bounding boxes
        scores: Tensor [N] de scores de confiance
        iou_threshold: Seuil IoU pour supprimer boxes chevauchantes
    
    Returns:
        indices: Indices des boxes à garder
    """
    # Trier par score décroissant
    sorted_indices = torch.argsort(scores, descending=True)
    
    keep = []
    
    while len(sorted_indices) > 0:
        # Garder la box avec le meilleur score
        current = sorted_indices[0]
        keep.append(current.item())
        
        if len(sorted_indices) == 1:
            break
        
        # Calculer IoU avec les autres boxes
        current_box = boxes[current].unsqueeze(0)
        other_boxes = boxes[sorted_indices[1:]]
        ious = box_iou(current_box, other_boxes)[0]
        
        # Garder seulement les boxes avec IoU < threshold
        mask = ious < iou_threshold
        sorted_indices = sorted_indices[1:][mask]
    
    return torch.tensor(keep)

# Exemple : Plusieurs détections pour le même objet
boxes = torch.tensor([
    [100, 100, 200, 200],  # Box 1
    [110, 110, 210, 210],  # Box 2 (très chevauchante avec 1)
    [120, 120, 220, 220],  # Box 3 (très chevauchante avec 1 et 2)
    [300, 300, 400, 400],  # Box 4 (objet différent)
], dtype=torch.float32)

scores = torch.tensor([0.9, 0.85, 0.7, 0.95])  # Scores de confiance

# Appliquer NMS
keep_indices = non_max_suppression(boxes, scores, iou_threshold=0.5)

print(f"Boxes avant NMS: {len(boxes)}")
print(f"Boxes après NMS: {len(keep_indices)}")
print(f"Indices gardés: {keep_indices.tolist()}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, title, show_all in zip(axes, ['Avant NMS', 'Après NMS'], [True, False]):
    ax.set_xlim(0, 500)
    ax.set_ylim(0, 500)
    ax.invert_yaxis()
    ax.set_aspect('equal')
    
    indices_to_show = range(len(boxes)) if show_all else keep_indices
    
    for i in indices_to_show:
        box = boxes[i]
        score = scores[i]
        
        color = 'green' if i in keep_indices else 'red'
        alpha = 1.0 if i in keep_indices else 0.3
        
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                                linewidth=2, edgecolor=color, facecolor='none', alpha=alpha)
        ax.add_patch(rect)
        ax.text(box[0], box[1]-5, f'Box {i} ({score:.2f})', color=color)
    
    ax.set_title(title)

plt.tight_layout()
plt.show()

## 2. Faster R-CNN avec torchvision

Faster R-CNN est un détecteur two-stage (RPN + détection).

In [ ]:
# Charger modèle pré-entraîné Faster R-CNN (COCO dataset)
model_frcnn = fasterrcnn_resnet50_fpn(pretrained=True)
model_frcnn.to(device)
model_frcnn.eval()

# Classes COCO (91 classes)
COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

print(f"Modèle chargé : Faster R-CNN ResNet50-FPN")
print(f"Nombre de classes : {len(COCO_CLASSES)}")

In [ ]:
def load_image_from_url(url):
    """Charge une image depuis une URL."""
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    return img

def detect_faster_rcnn(image, model, threshold=0.5):
    """
    Détecte des objets avec Faster R-CNN.
    
    Args:
        image: PIL Image
        model: Modèle Faster R-CNN
        threshold: Seuil de confiance minimum
    
    Returns:
        boxes, labels, scores
    """
    # Transformation
    transform = transforms.Compose([transforms.ToTensor()])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # Inférence
    start_time = time.time()
    with torch.no_grad():
        predictions = model(img_tensor)
    inference_time = time.time() - start_time
    
    # Extraire résultats
    pred = predictions[0]
    boxes = pred['boxes'].cpu()
    labels = pred['labels'].cpu()
    scores = pred['scores'].cpu()
    
    # Filtrer par threshold
    mask = scores > threshold
    boxes = boxes[mask]
    labels = labels[mask]
    scores = scores[mask]
    
    return boxes, labels, scores, inference_time

def visualize_detections(image, boxes, labels, scores, class_names):
    """Visualise les détections."""
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        
        # Rectangle
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1,
                                linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        
        # Label
        class_name = class_names[label]
        ax.text(x1, y1-5, f'{class_name} {score:.2f}',
               bbox=dict(facecolor='red', alpha=0.5), fontsize=10, color='white')
    
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Exemple : détecter dans une image
img_url = "https://ultralytics.com/images/bus.jpg"
image = load_image_from_url(img_url)

boxes, labels, scores, inference_time = detect_faster_rcnn(image, model_frcnn, threshold=0.7)

print(f"\n=== Faster R-CNN Résultats ===")
print(f"Temps d'inférence: {inference_time:.3f}s")
print(f"Nombre de détections: {len(boxes)}")
print(f"\nDétections:")
for i, (label, score) in enumerate(zip(labels, scores)):
    print(f"  {i+1}. {COCO_CLASSES[label]} (confiance: {score:.3f})")

visualize_detections(image, boxes, labels, scores, COCO_CLASSES)

## 3. YOLOv8 avec Ultralytics

YOLO est beaucoup plus rapide que Faster R-CNN (one-stage detector).

In [ ]:
if YOLO_AVAILABLE:
    # Charger YOLOv8 nano (le plus rapide)
    model_yolo = YOLO('yolov8n.pt')
    
    print("YOLOv8 nano chargé")
    print(f"Nombre de paramètres: {sum(p.numel() for p in model_yolo.model.parameters()) / 1e6:.1f}M")
else:
    print("YOLOv8 non disponible. Installer avec: pip install ultralytics")

In [ ]:
if YOLO_AVAILABLE:
    # Inférence avec YOLOv8
    results = model_yolo(img_url)
    
    # Afficher résultats
    result = results[0]
    
    print(f"\n=== YOLOv8 Résultats ===")
    print(f"Temps d'inférence: {result.speed['inference']:.1f}ms")
    print(f"Nombre de détections: {len(result.boxes)}")
    print(f"\nDétections:")
    
    for i, box in enumerate(result.boxes):
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        label = model_yolo.names[cls]
        print(f"  {i+1}. {label} (confiance: {conf:.3f})")
    
    # Visualisation (YOLOv8 a une fonction plot intégrée)
    result_img = result.plot()
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('YOLOv8 Détections')
    plt.show()

## 4. Comparaison Faster R-CNN vs YOLOv8

Comparons les performances en vitesse et précision.

In [ ]:
if YOLO_AVAILABLE:
    # Tester sur plusieurs images
    test_images = [
        "https://ultralytics.com/images/bus.jpg",
        "https://ultralytics.com/images/zidane.jpg",
    ]
    
    frcnn_times = []
    yolo_times = []
    
    print("Benchmark Faster R-CNN vs YOLOv8...\n")
    
    for url in test_images:
        img = load_image_from_url(url)
        
        # Faster R-CNN
        _, _, _, frcnn_time = detect_faster_rcnn(img, model_frcnn, threshold=0.7)
        frcnn_times.append(frcnn_time)
        
        # YOLOv8
        start = time.time()
        _ = model_yolo(img)
        yolo_time = time.time() - start
        yolo_times.append(yolo_time)
        
        print(f"Image: {url.split('/')[-1]}")
        print(f"  Faster R-CNN: {frcnn_time:.3f}s")
        print(f"  YOLOv8: {yolo_time:.3f}s")
        print(f"  Speedup: {frcnn_time/yolo_time:.1f}x\n")
    
    # Moyenne
    avg_frcnn = np.mean(frcnn_times)
    avg_yolo = np.mean(yolo_times)
    
    print(f"=== Moyennes ===")
    print(f"Faster R-CNN: {avg_frcnn:.3f}s ({1/avg_frcnn:.1f} FPS)")
    print(f"YOLOv8: {avg_yolo:.3f}s ({1/avg_yolo:.1f} FPS)")
    print(f"YOLOv8 est {avg_frcnn/avg_yolo:.1f}x plus rapide")
    
    # Visualisation
    fig, ax = plt.subplots(figsize=(10, 6))
    models = ['Faster R-CNN', 'YOLOv8']
    times = [avg_frcnn * 1000, avg_yolo * 1000]  # en ms
    fps = [1/avg_frcnn, 1/avg_yolo]
    
    x = np.arange(len(models))
    width = 0.35
    
    ax.bar(x - width/2, times, width, label='Temps (ms)', alpha=0.8)
    ax2 = ax.twinx()
    ax2.bar(x + width/2, fps, width, label='FPS', color='orange', alpha=0.8)
    
    ax.set_ylabel('Temps (ms)')
    ax2.set_ylabel('FPS')
    ax.set_xlabel('Modèle')
    ax.set_title('Comparaison Vitesse : Faster R-CNN vs YOLOv8')
    ax.set_xticks(x)
    ax.set_xticklabels(models)
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

## 5. Entraînement YOLOv8 sur Dataset Custom

Entraîner YOLOv8 sur un dataset au format COCO.

In [ ]:
if YOLO_AVAILABLE:
    # Configuration dataset (format YAML)
    dataset_config = """
# Dataset configuration for YOLOv8
path: /workspace/data/custom_detection  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')

# Classes
names:
  0: person
  1: car
  2: bike
"""
    
    # Sauvegarder config
    import yaml
    config_path = '/tmp/custom_dataset.yaml'
    with open(config_path, 'w') as f:
        f.write(dataset_config)
    
    print("Configuration dataset créée")
    print(dataset_config)

In [ ]:
if YOLO_AVAILABLE:
    # Entraînement (exemple - nécessite un dataset réel)
    # ATTENTION: Cette cellule nécessite un dataset au format YOLO
    
    # Structure attendue:
    # custom_detection/
    #   ├── images/
    #   │   ├── train/
    #   │   │   ├── img1.jpg
    #   │   │   └── img2.jpg
    #   │   └── val/
    #   │       ├── img1.jpg
    #   │       └── img2.jpg
    #   └── labels/
    #       ├── train/
    #       │   ├── img1.txt  # Format: <class> <x_center> <y_center> <width> <height> (normalized)
    #       │   └── img2.txt
    #       └── val/
    #           ├── img1.txt
    #           └── img2.txt
    
    print("""Pour entraîner YOLOv8 sur un dataset custom:
    
1. Préparer les données au format YOLO
2. Créer un fichier YAML de configuration
3. Lancer l'entraînement:

model = YOLO('yolov8n.pt')  # partir du modèle pré-entraîné
results = model.train(
    data='custom_dataset.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='yolov8_custom',
    patience=50,  # early stopping
    save=True,
    device=0  # GPU 0
)

4. Évaluer:

metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

5. Inférence:

model = YOLO('runs/detect/yolov8_custom/weights/best.pt')
results = model('path/to/image.jpg')
""")

## 6. Métriques d'Évaluation : mAP

Calculer la Mean Average Precision pour évaluer un détecteur.

In [ ]:
def compute_precision_recall(pred_boxes, pred_scores, gt_boxes, iou_threshold=0.5):
    """
    Calcule la courbe Precision-Recall.
    
    Args:
        pred_boxes: Tensor [N, 4] de prédictions
        pred_scores: Tensor [N] de scores
        gt_boxes: Tensor [M, 4] de ground truths
        iou_threshold: Seuil IoU pour considérer une détection correcte
    
    Returns:
        precisions, recalls
    """
    # Trier prédictions par score décroissant
    sorted_indices = torch.argsort(pred_scores, descending=True)
    pred_boxes = pred_boxes[sorted_indices]
    pred_scores = pred_scores[sorted_indices]
    
    n_gt = len(gt_boxes)
    tp = torch.zeros(len(pred_boxes))
    fp = torch.zeros(len(pred_boxes))
    
    detected_gt = set()
    
    for i, pred_box in enumerate(pred_boxes):
        # Calculer IoU avec tous les GT
        ious = box_iou(pred_box.unsqueeze(0), gt_boxes)[0]
        max_iou, max_idx = ious.max(0)
        
        if max_iou >= iou_threshold and max_idx.item() not in detected_gt:
            # True Positive
            tp[i] = 1
            detected_gt.add(max_idx.item())
        else:
            # False Positive
            fp[i] = 1
    
    # Calcul cumulatif
    tp_cumsum = torch.cumsum(tp, dim=0)
    fp_cumsum = torch.cumsum(fp, dim=0)
    
    # Precision et Recall
    precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-10)
    recalls = tp_cumsum / n_gt
    
    return precisions.numpy(), recalls.numpy()

def compute_ap(precisions, recalls):
    """Calcule Average Precision (aire sous courbe PR)."""
    # Ajouter points (0, 1) et (1, 0)
    precisions = np.concatenate([[0], precisions, [0]])
    recalls = np.concatenate([[0], recalls, [1]])
    
    # Interpolation
    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = max(precisions[i], precisions[i + 1])
    
    # Calcul aire (méthode des rectangles)
    indices = np.where(recalls[1:] != recalls[:-1])[0] + 1
    ap = np.sum((recalls[indices] - recalls[indices - 1]) * precisions[indices])
    
    return ap

# Exemple
pred_boxes = torch.tensor([
    [50, 50, 150, 150],
    [55, 55, 155, 155],
    [200, 200, 300, 300],
    [210, 210, 310, 310],
], dtype=torch.float32)

pred_scores = torch.tensor([0.9, 0.85, 0.8, 0.75])

gt_boxes = torch.tensor([
    [50, 50, 150, 150],
    [200, 200, 300, 300],
], dtype=torch.float32)

precisions, recalls = compute_precision_recall(pred_boxes, pred_scores, gt_boxes, iou_threshold=0.5)
ap = compute_ap(precisions, recalls)

print(f"Average Precision (AP@0.5): {ap:.3f}")

# Visualiser courbe PR
plt.figure(figsize=(8, 6))
plt.plot(recalls, precisions, marker='o', linewidth=2)
plt.fill_between(recalls, precisions, alpha=0.3)
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title(f'Precision-Recall Curve (AP = {ap:.3f})', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.tight_layout()
plt.show()

## 7. Détection Temps Réel sur Webcam

In [ ]:
if YOLO_AVAILABLE:
    print("""Pour utiliser YOLOv8 en temps réel sur webcam:

import cv2
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

cap = cv2.VideoCapture(0)  # Webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Détection
    results = model(frame)
    
    # Affichage
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv8 Detection', annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Ou encore plus simple:
model.predict(source=0, show=True)  # 0 = webcam
""")

## Résumé

Dans ce notebook, nous avons exploré :

1. **Fondamentaux** :
   - IoU (Intersection over Union) pour mesurer chevauchement
   - NMS (Non-Maximum Suppression) pour éliminer détections redondantes

2. **Faster R-CNN** (two-stage) :
   - RPN + RoI Pooling + classification/régression
   - Précis mais lent (~5 FPS)

3. **YOLOv8** (one-stage) :
   - Détection en une seule passe
   - Très rapide (~80 FPS) pour temps réel
   - API Ultralytics simple et moderne

4. **Métriques** :
   - Courbe Precision-Recall
   - Average Precision (AP)
   - Mean Average Precision (mAP)

5. **Applications** :
   - Détection d'objets dans images/vidéos
   - Entraînement sur datasets customs
   - Détection temps réel webcam

### Points Clés
- **Trade-off vitesse/précision** : Faster R-CNN (précis) vs YOLO (rapide)
- **Transfer learning** : toujours partir d'un modèle pré-entraîné (COCO)
- **mAP** : métrique standard pour évaluer détecteurs
- **NMS** : essentiel pour éliminer détections multiples

### Prochaines Étapes
- Notebook suivant : Segmentation sémantique (U-Net)
- Explorer d'autres architectures : RetinaNet, EfficientDet
- Appliquer à vos propres données